In [ ]:
import pandas as pd
import openpyxl
df = pd.read_excel('pi.xlsx')
df.rename(columns = {'Unnamed: 2':'key', 'Unnamed: 4':'value'}, inplace = True)
date_no = df.loc[df['key'] == 'DATE'].iloc[0]['value']
PI_no = df.loc[df['key'] == 'PROFORMA INVOICE NO.'].iloc[0]['value']

import smtplib
import ssl
import json
from datetime import datetime, timedelta

def send_email(sender_email, app_password, receiver_email, subject, body):
    port = 465  # For SSL
    context = ssl.create_default_context()

    with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
        server.login(sender_email, app_password)
        message = f"Subject: {subject}\n\n{body}"
        server.sendmail(sender_email, receiver_email, message)

def check_invoice_received(invoice_date):
    today = datetime.today()
    if today > (invoice_date + timedelta(days=30)):
        return False
    return True

def main():
    # Load configuration from file
    with open("config.json") as f:
        config = json.load(f)

    # Replace with the actual extracted values from the table
    date = date_no
    pi_number = PI_no

    sender_email = config["sender_email"]
    app_password = config["app_password"]
    receiver_email = config["receiver_email"]
    subject = "Invoice Alert PI {}".format(pi_number)

    invoice_date = datetime.strptime(date, "%d/%m/%Y")
    invoice_received = check_invoice_received(invoice_date)

    if not invoice_received:
        body = f"The Proforma Invoice {pi_number} has not been received within 30 days."
        send_email(sender_email, app_password, receiver_email, subject, body)

if __name__ == "__main__":
    main()